In [1]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("comments")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [4]:
query_popular = """
            SELECT parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            HAVING COUNT(id) > 10
            """

In [5]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

popular_comments = query_job.to_dataframe()
popular_comments.head()

,parent,f0_
0,6126926,49
1,780569,40
2,2772387,104
3,6930109,39
4,1820561,46


In [6]:
popular_comments.columns

Index(['parent', 'f0_'], dtype='object')

In [8]:
popular_comments.sort_values(by='f0_', ascending=False)

,parent,f0_
2772,363,1311
3981,9812245,902
49,9996333,850
4266,9303396,785
4132,10152809,733
...,...,...
12266,1530803,11
12265,5108218,11
12264,10029272,11
12263,7201263,11


In [11]:
TEN_GB = 10**10
def get_query_result(my_query):
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=TEN_GB)
    query_job = client.query(my_query, job_config=safe_config)
    
    my_res = query_job.to_dataframe()
    return my_res

In [13]:
query_res = get_query_result(query_popular)
query_res.head()

,parent,f0_
0,6126926,49
1,780569,40
2,2772387,104
3,6930109,39
4,1820561,46


In [14]:
query_popular_2 = """
            SELECT parent, COUNT(1) AS NumPosts
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            HAVING COUNT(id) > 10
            """

In [15]:
query_res = get_query_result(query_popular_2)
query_res.head()

,parent,NumPosts
0,6683866,39
1,6627329,46
2,3476843,49
3,7234010,48
4,2932956,76


In [23]:
query_3 = """
            SELECT parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            HAVING COUNT(id) > 50
            """

In [24]:
query_res = get_query_result(query_3)
query_res.head()

,parent,f0_
0,8120079,148
1,7075537,51
2,5511466,83
3,9905363,75
4,4596375,268


In [25]:
len(query_res)

2401

## Exercise

In [26]:
query_3 = """
            SELECT author, COUNT(1) AS NumPosts
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY author
            HAVING COUNT(1) > 10000
        """

In [27]:
query_res = get_query_result(query_3)
query_res.head()

,author,NumPosts
0,DanBC,12902
1,sp332,10882
2,davidw,10764
3,rayiner,11080
4,tptacek,33839


In [30]:
table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [35]:
query_4 = """
            SELECT COUNT(1) AS NumPosts
            FROM `bigquery-public-data.hacker_news.comments`
            WHERE deleted = True
        """

In [36]:
query_res = get_query_result(query_4)
query_res.head()

,NumPosts
0,227736


In [37]:
query_res.NumPosts[0]

227736